In [ ]:
# @title УСТАНОВКА ЗАВИСИМОСТЕЙ
%%capture
!pip install gdown
!pip install mediapipe

In [ ]:
# @title ПОДКЛЮЧЕНИЕ МОДУЛЕЙ
import os
import sys
import cv2
import csv
import math
import gdown
import shutil
import numpy as np
import pandas as pd
import mediapipe as mp
from moviepy.editor import VideoFileClip
from google.colab.patches import cv2_imshow
from google.colab import drive



In [ ]:
print('mediapipe:', mp.__version__)

mediapipe: 0.10.3


In [ ]:
# @title ОПРЕДЕЛЕНИЕ ПЕРЕМЕННЫХ ПУТИ К КАТАЛОГАМ И ФАЙЛАМ
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks'
# Путь до каталога с каталогами видов ударов
path_data_videos = f'{path}/УИИ/3. Стажировка/pose_estimate/3.1 MediaPipe/Каратэ/Karate_Project/Karate_Petrov'

Mounted at /content/drive


In [ ]:
# @title ОПРЕДЕЛЕНИЕ КЛАССА КАРАТЭ
class Model_Karate:
    # КОНСТРУКТОР КЛАССА
    def __init__(self,
                 name='Karate_moves',   # Название проекта
                 root_dir='/content',   # Базовый каталог для корневого каталога проекта
                 data_dir='',           # Каталог с исходными видео
                 framework='midiapipe'  # Фреймворк для создания скелета
                 ):

        #======================================
        # ОПРЕДЕЛЕНИЕ ПАРАМЕТРОВ ПРОЕКТА
        #======================================
        # Вид движений
        self.name = name
        # Фреймворк
        self.framework = framework
        # Директория хранения датасета с видео
        self.directory_data = data_dir
        #======================================
        # СОЗДАНИЕ АРХИТЕКТУРЫ ПРОЕКТА
        #======================================
        # Директория для создания проекта
        self.directory_path = f'{root_dir}/{self.name}'

        if os.path.exists(self.directory_path):
            # Если директория существует, спрашиваем пользователя о удалении
            choice = input(f"Директория '{self.directory_path}' уже существует. Хотите удалить её рекурсивно? (y/n): ")
            if choice.lower() == 'y':
                # Удаляем директорию рекурсивно, если пользователь согласился
                try:
                    shutil.rmtree(self.directory_path)
                    print(f"Директория '{self.directory_path}' успешно удалена.")

                    os.makedirs(self.directory_path)
                    print(f"Директория '{self.directory_path}' успешно создана.")

                except OSError as e:
                    print(f"Ошибка при удалении директории: {e}")
        else:
            # Создаем новую директорию
            try:
                os.makedirs(self.directory_path)
                print(f"Директория '{self.directory_path}' успешно создана.")
            except OSError as e:
                print(f"Ошибка при создании директории: {e}")

        #======================================
        # СОЗДАНИЕ СКЕЛЕТА (пока midiapipe)
        #======================================
        if self.framework == 'midiapipe':
            print(f'framework: {self.framework}\nversion: {mp.__version__}')
            # Объявляем, что хотим использовать глобальную переменную mp_pose
            global mp_pose
            mp_pose = mp.solutions.pose

            # import mediapipe as mp
            global mp_drawing
            mp_drawing = mp.solutions.drawing_utils # Drawing helpers

            global mp_holistic
            mp_holistic = mp.solutions.holistic # Mediapipe Solutions
        #======================================
        # Список точек для углов(угол относительно центральной)
        #======================================
        self.angles_Igor_Rybin = [[32, 28, 26], # Стопы
                                  [31, 27, 25],
                                  [28, 26, 24], # Колени
                                  [27, 25, 23],
                                  [26, 24, 23], # Пах
                                  [25, 23, 24],
                                  [26, 24, 12], # Изгиб тела
                                  [25, 23, 11],
                                  [24, 12, 14], # Плечи
                                  [23, 11, 13],
                                  [12, 14, 16], # Локти
                                  [11, 13, 15],
                                  [11, 12, 5], # Глаза
                                  [12, 11, 2]
                                  ]
        #======================================
        print(self.angles_Igor_Rybin)

    # Функция-парсер для имени файла на КЛАССЫ!
    def parse_filename(self, filename):
        '''
        filename = 'UDAR_VID_POYAS_CHELOVEK_NUMERVIDEO.mp4'
        '''
        file_name = os.path.splitext(os.path.basename(filename))[0]  # Удаление расширения файла
        parts = file_name.split('_')  # Разделение имени файла по символу "_"

        class_name = parts[0]  # Значение класса удара
        class_view = parts[1]  # Значение класса ракурса камеры
        class_belt = parts[2]  # Значение класса цвета пояса
        karateka = parts[3]    # Значение класса исполнителя удара
        num_video = parts[4]   # Значение класса номера видео

        return class_name, class_view, class_belt, karateka, num_video

    # Функция для вычисления углов
    def calculate_triangle_angles(self, point1, point2, point3):
        # Вычисление длин сторон треугольника
        side_a = math.dist(point1, point2)
        side_b = math.dist(point2, point3)
        side_c = math.dist(point3, point1)

        # Вычисление углов треугольника с помощью формулы косинусов
        angle_A = math.degrees(math.acos((side_c**2 + side_a**2 - side_b**2) / (2 * side_c * side_a)))
        angle_B = math.degrees(math.acos((side_a**2 + side_b**2 - side_c**2) / (2 * side_a * side_b)))
        angle_C = math.degrees(math.acos((side_b**2 + side_c**2 - side_a**2) / (2 * side_b * side_c)))

        return angle_A, angle_B, angle_C, (side_a, side_b, side_c)

    # Функция создания файла-отчета
    def report_to_file(self, file_report, text):

        with open(file_report, 'a') as file:
            file.write(text + '\n')

    # Функция подсчета полного числа файлов в каталоге и подкаталогах
    def count_files(self, path):
        count = 0
        for root, dirs, files in os.walk(path):
            count += len(files)
        return count

    def Video_to_DataFrame_to_CSV_Angles_clean(self,
                                               file_name_video_in,     # Путь к входному видео
                                               file_name_csv_angles,   # Путь к CSV Для углов (при обучении лучше сначала нормализовать)
                                               list_angles,            # Список углов
                                               file_name_video_out='video_out.avi', # Путь к выходному видео
                                               ):

        print('list_angles', list_angles)

        # Значения по умолчанию, чтобы не разваливалась обработка (костыль)
        class_name='Default', # Вид удара
        class_belt='Default', # Цвет пояса
        class_view='Default', # Положение камеры
        karateka='Default', # Номер исполнителя с определенным цветом пояса
        num='Default' # Номер видео

        # Метаданные об объекте исследования
        metadata = ['class']+['view']+['belt']+['karateka']+['num']+['frame']

        # Характеристики модели, описывающей объект исследования
        landmarks = []

        # Определяем количество кадров в видеофайле
        video = VideoFileClip(file_name_video_in)
        count_frames = int(video.reader.nframes)

        print("---------------------------------------")
        print('Количество кадров в видеофайле:', count_frames)

        # Получаем имена классов из названия видео (вид удара, цвет пояса, вид камеры и намер видео (если нужен)):
        classes = self.parse_filename(file_name_video_in)
        class_name = classes[0]
        class_view =  classes[1]
        class_belt =  classes[2]
        karateka = classes[3]
        num = classes[4]
    #--------------------------------------------------
        print("---------------------------------------")
        print(f"class_name: {class_name}")
        print(f"class_view: {class_view}")
        print(f"class_belt: {class_belt}")
        print(f"karateka: {karateka}")
        print(f"num_video: {num}")
    #--------------------------------------------------
        cap = cv2.VideoCapture(file_name_video_in)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
    #--------------------------------------------------
        print("---------------------------------------")
        print(f"Ширина кадра: {frame_width}")
        print(f"Высота кадра: {frame_height}")
        print(f"Частота кадров: {fps}")
    #--------------------------------------------------

        # angles = [[13, 11, 23], [14, 12, 24], [11, 13, 15], [12, 14, 16], [24, 23, 25], [23, 24, 26], [23, 25, 27], [24, 26, 28]]
        angles = list_angles

        print('angles:', angles)
        print('len(angles)', len(angles))

        # Список из углов для заголовка в таблицу (enumerate(angles) позволяет получить одновременно счетчик и значение из списка)
        # ccolumns = [f'ang_{i}' for i in range(len(angles))]
        columns = [f'ang_{i}_({col[1]})' for i, col in enumerate(angles)]
        # print(columns)
        print('metadata + columns', metadata + columns)


        # fourcc = cv2.VideoWriter_fourcc(*'XVID')
        # out = cv2.VideoWriter(file_name_video_out, fourcc, fps, (frame_width, frame_height))

        with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            # Счетчик кадров
            num_frame = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                            )

                try:
                    pose = results.pose_landmarks.landmark
                    # pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    #-------------------------------------------------
                    angle_values = []

                    for angle_list in angles:
                        i, j, k = angle_list
                        Point_First = [pose[i].x, pose[i].y, pose[i].z]
                        Point_Mid = [pose[j].x, pose[j].y, pose[j].z]
                        Point_End = [pose[k].x, pose[k].y, pose[k].z]
                        # calc_angle = calculate_angle(Point_First, Point_Mid, Point_End)

                        print(i, j, k)
                        print('Point_First:', Point_First)
                        print('Point_Mid:', Point_Mid)
                        print('Point_End:', Point_End)

                        # Углы между точками: (82.60309475396538, 50.46450114357366, 46.932404102460964) градусов
                        calc_angle = self.calculate_triangle_angles(Point_First, Point_Mid, Point_End)

                        angle_values.append(calc_angle[1])

                    class_values = []
                    print('angle_values_f', angle_values)
                    [class_values.append(cl) for cl in [classes[0], classes[1],classes[2], classes[3], classes[4], num_frame]]
                    print('class_values', class_values)
                    print('len-class_values+angle_values', len(class_values+angle_values))
                    print('class_values+angle_values', class_values+angle_values)
                    # new_row = pd.Series(class_values+angle_values, index=df_ang.columns)
                    # print('new_row', new_row)
                    # df_ang = df_ang.append(new_row, ignore_index=True)
                    print('metadata + columns', metadata + columns)

                    with open(file_name_csv_angles, mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        if f.tell() == 0:  # Check if file is empty

                            csv_writer.writerow(metadata + columns)
                        csv_writer.writerow(class_values+angle_values)

                except Exception as e:
                    print(f"Произошла ошибка: {e}")

                # Запись в видео
                # out.write(image)

                # Вывод на изображения в формате cv2 в colab
                # cv2_imshow(image)


                '''
                # Чтобы избежать дозаписывания, удаляем файл, если он существовал ранее
                if os.path.exists(file_name_csv_angles):
                    # Файл существует, удаляем его
                    os.remove(file_name_csv_angles)
                    print(f"Удален существующий ранее файл: {file_name_csv_angles}")
                '''
                # df_ang.to_csv(file_name_csv_angles, index=False)

    # Таким образом, новые данные из df_ang будут дозаписываться в конец файла CSV, без перезаписи уже существующих данных.
    # Обратите внимание, что параметр header установлен в False, чтобы не записывать заголовок (шапку) в файл при каждой дозаписи.
                '''
                            # Открываем файл CSV в режиме дозаписи
                            with open(file_name_csv_angles, mode='a', newline='') as file:
                                # Создаем объект csv.writer
                                writer = csv.writer(file)

                                # Записываем данные в конец файла
                                df_ang.to_csv(writer, index=False, header=False)
                '''



        # Основные параметры видео
                report_info = [count_frames,  # Количество кадров в видеофайле
                              class_name,    # class_name
                              class_view,    # class_view
                              class_belt,    # class_belt
                              karateka,      # karateka
                              num,           # num_video
                              frame_width,   # Ширина кадра
                              frame_height,  # Высота кадра
                              fps            # Частота кадров
                              ]

                num_frame +=1

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        # df_ang.to_csv(file_name_csv_angles, index=False)

        return report_info

        cap.release()
        cv2.destroyAllWindows()

    #Функция обхода файлов в каталоге и вложенных подкаталогах
    def process_folder_angles(self,
                              folder_path, # Корневая папка с видео файлами
                              coords_csv,  # Фаил с результатами обработки видео
                              angles=None, # Список углов
                              file_report = 'test.txt'):  # Файл для записи отчета о процессе обработки

        # Список углов для датасета
        if angles is None:
            angles = self.angles_Igor_Rybin

        # Функция подсчета файлов в каталоге
        total_files_folder = self.count_files(folder_path)
        print(f"Общее количество файлов в {os.path.basename(folder_path)}:", total_files_folder)

        # Если писать в отдельный файл, то существующий с таким названием будет удален
        if os.path.exists(file_report):
            # Файл существует, удаляем его
            os.remove(file_report)
            print(f"Удален существующий ранее файл: {file_report}")

        # Пишем в файл отчета информацию о количестве файлов в каталоге
        with open(file_report, 'a') as file:
            text = f'''
=======================================
 Общее число файлов в каталоге {os.path.basename(folder_path)}: {total_files_folder}
=======================================
            '''
            file.write(text + '\n')

        # Если писать в отдельный файл, то существующий с таким названием будет удален
        if os.path.exists(coords_csv):
            # Файл существует, удаляем его
            os.remove(coords_csv)
            print(f"Удален существующий ранее файл: {coords_csv}")

        # Счетчик файлов
        num_file = 0

        for root, dirs, files in os.walk(folder_path):
            # print(root) # Корневая папка (содержит ссылку (ярлык))
            print(dirs) # Cписок директорий внутри
            # print(files) # Список файлов внутри (здесь их нет)

            # Обработка каталогов
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                pass

            # Обработка файлов
            for file in sorted(files):
                file_path = os.path.join(root, file)

                print("---------------------------------------")
                print(f'Имя файла: {file}')

                # Код для обработки файла (результат - список параметров)
                report_info = self.Video_to_DataFrame_to_CSV_Angles_clean(file_path, coords_csv, angles)

                print("---------------------------------------")
                print('Обработка прошла успешно!')

                num_file +=1

                # Счетчик обработанных файлов!
                print("---------------------------------------")
                print(f'Обработано: {num_file} файл(ов)')

                # Текст отчета (при ''' - строки сохраняют форматирование как есть)
                text_to_append = f'''
---------------------------------------
Имя файла: {file}
---------------------------------------
Количество кадров в видеофайле: {report_info[0]}
---------------------------------------
class_name: {report_info[1]}
class_view: {report_info[2]}
class_belt: {report_info[3]}
karateka: {report_info[4]}
num_video: {report_info[5]}
---------------------------------------
Ширина кадра: {report_info[6]}
Высота кадра: {report_info[7]}
Частота кадров: {report_info[8]}
---------------------------------------
Обработано: {num_file} файл(ов)
---------------------------------------
                '''
                # text_to_append = 'Привет, мир!'
                # Запись параметров обработанных видео в файл отчета.
                try:
                    print('file_report:', file_report)
                    self.report_to_file(file_report, text_to_append)
                    print('Запись прошла успешно!')
                except Exception as e:
                    print(f'Случилось: {e}')

    # Обход каталогов поштучно в директории (Отчет...) и создание общего датафрейма!
    def All_Dataset(self,
                    folder_csv,
                    dir_report_df):

        # dir_report_df = f'{path_proj}/Отчет_и_данные_CSV_Angles_версия_1'

        # Получение списка файлов в заданном каталоге с расширением .csv
        csv_files = [file for file in os.listdir(folder_csv) if file.endswith('.csv')]

        # Создание пустого DataFrame для объединения данных
        merged_df = pd.DataFrame()

        # # Чтение и объединение файлов CSV порядку
        for file in sorted(csv_files):
            print(file)

            file_path = os.path.join(folder_csv, file)  # Полный путь к файлу
            df = pd.read_csv(file_path)  # Чтение файла CSV в DataFrame
            merged_df = pd.concat([merged_df, df])  # Объединение DataFrame

        # Запись DataFrame в файл CSV
        merged_df.to_csv(f'{dir_report_df}/All_DataSet_Angles_v2_beta.csv', index=False)

    # ФУНКЦИЯ СДЕЛАТЬ ВСЁ (УГЛЫ)
    def Create_Dataset_Angles(self,
                              list_folders,
                              path_proj=None,
                              dir_video=None,
                              dir_current_csv=None
                              ):

          if path_proj is None:
              path_proj = self.directory_path

          if dir_video is None:
              dir_video = self.directory_data

          if dir_current_csv is None:
              # dir_current_csv = self.directory_path
              dir_current_csv = f'{self.directory_path}/Current_csv_class'

          #-------------------------------------------------------------
          try:
              # Создание директории
              os.makedirs(dir_current_csv)
          except:
              pass
          #-------------------------------------------------------------

          for dn in list_folders:
          # for dn in list_folders:

              # Путь к вложенному каталогу с видео
              path_to_video  = f'{dir_video}/{dn}'
              #-------------------------------------------------------------
              # Счетчик каталогов:
              class_folder = dn.split('-')[0]
              print(class_folder)
              #-------------------------------------------------------------
              # Файл для занесения результатов обработки видео
              file_name_csv = f'{dir_current_csv}/{class_folder}_pose_Angle_full_frame.csv'
              print(file_name_csv)
              #-------------------------------------------------------------
              # Файл отчета об обработанных видео
              file_path_report = f'{dir_current_csv}/{class_folder}_Отчет_о_записи_Углов.txt'
              print(file_path_report)
              #-------------------------------------------------------------
              print(path_to_video)
              self.process_folder_angles(path_to_video,
                                         file_name_csv,
                                         # angles_Igor_Rybin,
                                         file_report=file_path_report)

          dir_root = self.directory_path
          print(dir_root)
          self.All_Dataset(dir_current_csv, dir_root)


In [ ]:
karate_angle = Model_Karate(name='Karate_moves',
                            root_dir='/content',
                            data_dir=path_data_videos,
                            framework='midiapipe'
                            )

Директория '/content/Karate_moves' успешно создана.
framework: midiapipe
version: 0.10.3
[[32, 28, 26], [31, 27, 25], [28, 26, 24], [27, 25, 23], [26, 24, 23], [25, 23, 24], [26, 24, 12], [25, 23, 11], [24, 12, 14], [23, 11, 13], [12, 14, 16], [11, 13, 15], [11, 12, 5], [12, 11, 2]]


In [ ]:
# karate_angle.Create_Dataset_Coords()

In [ ]:
# @title Список каталогов в директории (Karate)

# path_proj = f'{path}/УИИ/3. Стажировка/pose_estimate/3.1 MediaPipe/Каратэ/Karate_Project'
# dir = f'{path_proj}/Karate'
dir = f'{path_data_videos}'

# Сортировка списка каталогов в алфавитном порядке
sorted_directories = sorted(os.listdir(dir))
print(os.listdir(dir))
print(sorted_directories)

['01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)', '03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)', '05-МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)', '06-МАЙ ГЕРИ КЕАГЕ (ПРЯМОЙ УДАР ЗАДНЕЙ НОГОЙ)', '07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)', '09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)']
['01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)', '03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)', '05-МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)', '06-МАЙ ГЕРИ КЕАГЕ (ПРЯМОЙ УДАР ЗАДНЕЙ НОГОЙ)', '07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)', '09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)']


In [ ]:
# for dn in sorted_directories:
list_folders_test = ['01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)']
list_folders1 = ['01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)', '03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)', '05-МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)']
list_folders2 = ['06-МАЙ ГЕРИ КЕАГЕ (ПРЯМОЙ УДАР ЗАДНЕЙ НОГОЙ)', '07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)', '09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)']
list_folders3 = ['08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)', '09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)', '10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)']

# karate_angle.Create_Dataset_Angles(list_folders_test, path_proj, dir)
karate_angle.Create_Dataset_Angles(list_folders_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
Point_End: [0.5929294228553772, 0.4648638069629669, -0.2484392523765564]
23 11 13
Point_First: [0.5329596400260925, 0.552809476852417, 0.08845924586057663]
Point_Mid: [0.5216405391693115, 0.37316998839378357, 0.09847133606672287]
Point_End: [0.5058318376541138, 0.4512878656387329, 0.2592882513999939]
12 14 16
Point_First: [0.5319068431854248, 0.386365681886673, -0.22238807380199432]
Point_Mid: [0.5929294228553772, 0.4648638069629669, -0.2484392523765564]
Point_End: [0.6418786644935608, 0.44301581382751465, -0.21742139756679535]
11 13 15
Point_First: [0.5216405391693115, 0.37316998839378357, 0.09847133606672287]
Point_Mid: [0.5058318376541138, 0.4512878656387329, 0.2592882513999939]
Point_End: [0.5066692233085632, 0.4602546989917755, 0.404246985912323]
11 12 5
Point_First: [0.5216405391693115, 0.37316998839378357, 0.09847133606672287]
Point_Mid: [0.5319068431854248, 0.386365681886673, -0.22238807380199432]
Point_End: [0

In [ ]:
df = pd.read_csv('Karate_moves/All_DataSet_Angles_v2_beta.csv')
df.head(len(df))

,class,view,belt,karateka,num,frame,ang_0_(28),ang_1_(27),ang_2_(26),ang_3_(25),ang_4_(24),ang_5_(23),ang_6_(24),ang_7_(23),ang_8_(12),ang_9_(11),ang_10_(14),ang_11_(13),ang_12_(12),ang_13_(11)
0,1,f,bb,1,1,0,70.380110,98.557500,146.714816,151.836591,108.169678,107.466511,153.759033,159.594171,4.528039,25.884944,102.093255,108.337245,25.279251,130.336577
1,1,f,bb,1,1,1,72.837685,100.546930,147.543250,148.912772,106.111729,112.345981,156.674050,153.479396,2.946224,25.741988,105.181700,105.409481,24.048843,130.911614
2,1,f,bb,1,1,2,73.057840,103.266461,149.389197,148.497770,106.318510,114.222315,157.074690,151.619954,6.785309,24.279891,107.950172,103.644868,23.855189,131.107008
3,1,f,bb,1,1,3,68.873606,100.180777,155.251267,145.998452,116.056114,113.009169,154.951891,147.531303,16.995008,25.210894,109.702268,103.157133,24.860751,130.379666
4,1,f,bb,1,1,4,67.114076,96.764812,154.514611,144.311978,117.342284,111.519383,151.607819,151.554661,9.303627,27.452859,103.683285,108.106207,25.057106,130.270680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11619,2,p,wb,1,10,43,63.224596,77.507572,102.425564,105.604924,116.290351,95.126864,137.386697,136.448916,61.318979,53.432230,123.407884,111.895308,97.640087,27.959326
11620,2,p,wb,1,10,44,65.844882,75.660487,103.510200,115.017957,118.902343,103.234675,132.488280,138.209359,71.163163,53.449981,127.506096,107.800345,99.819501,27.172478
11621,2,p,wb,1,10,45,66.848958,73.074225,106.080047,111.535379,119.209581,97.841424,134.886231,137.420923,68.083117,46.975017,124.733992,94.502837,100.482240,27.732428
11622,2,p,wb,1,10,46,66.254234,74.190373,106.274150,111.038558,119.335884,97.037166,134.508913,136.738649,66.972202,47.843837,125.878371,98.150603,106.520338,26.141690
